In [30]:
#! pip install tqdm

In [31]:
import numpy as np
import yaml  # or json
from pathlib import Path
import csv
import matplotlib.pyplot as plt
from multiprocessing import Pool, shared_memory
import os
from tqdm import tqdm 




### **Synthetic Population Generation via Categorical Constraint Matching**

#### **Core Methodology**
This implementation transforms raw census constraints and individual microdata into optimized numerical representations for synthetic population generation, using a three-stage process:

1. **Constraint Processing**  
   - Ingests structured CSV constraints (e.g., `age.csv`, `sex.csv`)  
   - Generates:  
     - A **label list** (e.g., `['age%16-24', 'age%25-34', 'sex%m', 'sex%f']`)  
     - A **target matrix** of census counts per geography zone (`np.array` shape: `[n_zones, n_categories]`)  

2. **Microdata Encoding**  
   - Converts individual records (`microdata.csv`) into a sparse binary matrix where:  
     - Rows = Individuals  
     - Columns = Constraint categories  
     - Values = `1` (matches category) or `0` (no match/missing)  
   - *Example*: An individual with `age=25-34` and `sex=m` encodes as `[0,1,0,0,0,0,1,0]`  

3. **Memory-Efficient Design**  
   - Uses pure NumPy arrays (no Pandas) for:  
     - Zero-copy sharing in multiprocessing  
     - O(1) incremental updates during annealing  
   - Handles missing data implicitly via zero-padding  

#### **Key Innovations**  
- **Deterministic Labeling**: Human-readable category prefixes (`age%`, `sex%`) ensure traceability  
- **Sparse-by-Design**: Binary encoding minimizes memory overhead  
- **Annealing-Ready**: Optimized for rapid constraint violation checks during optimization  

#### **Technical Highlights**  
```python
# Pseudocode of data flow
constraint_labels, constraint_targets = build_constraint_arrays(config)  # From age/sex CSVs
microdata_encoded = encode_microdata(config, constraint_labels)  # Binary matrix

# During annealing:
current_error = calculate_error(microdata_encoded, constraint_targets)  # L1/Chi-squared
```

---

### **Visualization of Data Flow**  
```mermaid
graph LR
    A[age.csv] --> C[Constraint Processor]
    B[sex.csv] --> C
    C --> D[Constraint Labels]
    C --> E[Target Matrix]
    F[microdata.csv] --> G[Microdata Encoder]
    D --> G
    G --> H[Binary Encoded Matrix]
```

---

### **Why This Works**  
- **Scalability**: Processes 100K+ individuals with minimal memory  
- **Flexibility**: New constraints require only YAML updates (no code changes)  
- **Reproducibility**: Explicit category mapping avoids hidden assumptions  



## Example YAML Configuration

```yaml
# Required microdata source
microdata:
  file: "data/microdata.csv"  # Path to individual records
  id_column: "ID"            # Optional unique identifier column

# List of constraint definitions
constraints:
  # Age distribution constraints
  - file: "data/age.csv"             # Census data file
    microdata_id: "Age"              # Matching column in microdata
    constraint_prefix: "Age%"        # Label prefix for categories
    geography_column: "GEO_CODE"    # Zone identifier column

  # Sex distribution constraints  
  - file: "data/sex.csv"
    microdata_id: "Sex"
    constraint_prefix: "Sex%"
    geography_column: "GEO_CODE"

In [32]:
def load_config(config_path):
    """Load YAML config file and validate structure."""
    config_path = Path(config_path)
    with open(config_path) as f:
        if config_path.suffix == '.yaml':
            config = yaml.safe_load(f)
        else:
            import json
            config = json.load(f)
    
    # Validate config structure
    assert "microdata" in config, "Config missing 'microdata' section"
    assert "constraints" in config and len(config["constraints"]) > 0, "No constraints defined"
    return config

## `load_config(config_path)`

**Purpose**:  
Loads and validates a configuration file (YAML or JSON) that defines the microdata and constraints structure.

**Inputs**:
- `config_path` (str or Path): Path to the configuration file (`.yaml` or `.json`)

**Returns**:
- `dict`: Parsed configuration with keys `'microdata'` and `'constraints'`

**Key Features**:
- Automatically detects file format (YAML/JSON) from extension
- Validates presence of required sections:
  - `'microdata'`: File path for microdata CSV
  - `'constraints'`: List of constraint definitions
- Raises `AssertionError` if structure is invalid

**Example Config**:
```yaml
microdata:
  file: "data/microdata.csv"
constraints:
  - file: "data/age.csv"
    microdata_id: "age"
    constraint_prefix: "age%"
    geography_column: "GEO_CODE"
    set_as_population_total: true

In [33]:
config = load_config('testdata/config.yaml')
print(config)

{'microdata': {'file': 'testdata/microdata.csv', 'id_column': 'ID'}, 'constraints': [{'file': 'testdata/age.csv', 'microdata_id': 'Age', 'constraint_prefix': 'Age%', 'geography_column': 'GEO_CODE', 'set_as_population_total': True}, {'file': 'testdata/sex.csv', 'microdata_id': 'Sex', 'constraint_prefix': 'Sex%', 'geography_column': 'GEO_CODE', 'set_as_population_total': False}]}


In [34]:
def build_constraint_arrays(config):
    """
    Enhanced version that:
    1. Uses set_as_population_total to calculate population sizes
    2. Tracks geography codes (GEOIDs) separately
    3. Returns results in a structured dict
    
    Returns:
        {
            "constraint_labels": List[str],
            "constraint_targets": np.array,
            "geography_codes": List[str],
            "population_totals": np.array
        }
    """
    constraint_labels = []
    constraint_targets = None
    geography_codes = []
    pop_total_constraint = False
    population_totals = []

    for constraint in config["constraints"]:
        with open(constraint["file"], mode='r') as f:
            reader = csv.reader(f)
            headers = next(reader)
            data = list(reader)
        
        poptotal_constraint = constraint["set_as_population_total"]
        print(poptotal_constraint)
            
        geo_col = constraint["geography_column"]
        geo_idx = headers.index(geo_col)
        
        # Store GEOIDs on first pass
        if not geography_codes:
            geography_codes = [row[geo_idx] for row in data]
        
        # Handle population totals if specified
        if pop_total_constraint: 
            population_totals = np.array([float(row[pop_idx]) for row in data])
        
        # Process categories
        categories = [h for i, h in enumerate(headers) if i != geo_idx]
        prefix = constraint["constraint_prefix"]
        constraint_labels.extend(f"{prefix}{cat}" for cat in categories)
        
        # Extract targets
        target_rows = []
        for row in data:
            target_values = [float(row[i]) for i in range(len(headers)) if i != geo_idx]
            target_rows.append(target_values)
            if poptotal_constraint:
                population_totals.append(sum(target_values))
        
        targets = np.array(target_rows)
        constraint_targets = targets if constraint_targets is None else np.hstack([constraint_targets, targets])
    
    return {
        "constraint_labels": constraint_labels,
        "constraint_targets": constraint_targets,
        "geography_codes": geography_codes,
        "population_totals": population_totals 
    }

In [35]:
constraints_dict = build_constraint_arrays(config)

True
False


In [36]:
constraints_dict

{'constraint_labels': ['Age%16-24',
  'Age%25-34',
  'Age%35-44',
  'Age%45-54',
  'Age%55-64',
  'Age%65-74',
  'Sex%m',
  'Sex%f'],
 'constraint_targets': array([[ 1350.,  1531.,  2086.,  2443.,  2304.,  1631.,  5572.,  5773.],
        [ 1719.,  2423.,  2472.,  2713.,  2457.,  1638.,  6683.,  6739.],
        [ 1694.,  1826.,  2480.,  2875.,  2446.,  1811.,  6372.,  6760.],
        [ 1997.,  2287.,  2218.,  2250.,  1645.,  1069.,  5652.,  5814.],
        [ 2789.,  3535.,  2824.,  1982.,  1533.,   947.,  6868.,  6742.],
        [ 3124.,  3740.,  2839.,  1795.,  1314.,   704.,  6769.,  6747.],
        [ 6715.,  5114.,  2725.,  1534.,  1154.,   668., 10125.,  7785.],
        [ 1931.,  2194.,  2364.,  2176.,  1852.,  1058.,  5434.,  6141.],
        [ 1392.,  1677.,  2211.,  2480.,  2335.,  1849.,  5764.,  6180.],
        [ 2124.,  2607.,  2283.,  2449.,  1899.,  1178.,  5976.,  6564.],
        [ 2196.,  2845.,  2369.,  1936.,  1628.,   918.,  5839.,  6053.],
        [ 2198.,  2679.,  2334

## `build_constraint_arrays(config)`

**Purpose**:  
Transforms constraint CSV files (like age/sex distributions) into labeled NumPy arrays for synthetic population generation.

### Inputs
- `config` (dict): Configuration dictionary containing:
  - `constraints`: List of constraint definitions (file paths, prefixes, etc.)

### Returns
- `constraint_labels` (List[str]): Formatted category labels  
  Example: `["age%16-24", "age%25-34", "sex%m", "sex%f"]`
- `constraint_targets` (np.array): 2D array of census counts  
  Shape: `(n_geographies, n_constraints)`

### Key Features
-  **File Processing**:
  - Reads CSV files without Pandas (vanilla Python `csv` module)
  - Handles header rows and geography columns intelligently
-  **Smart Labeling**:
  - Combines constraint prefixes with category names  
    (e.g., `"age%" + "25-34" → "age%25-34"`)
-  **Array Construction**:
  - Builds a consolidated NumPy array by horizontally stacking constraints
  - Automatically converts string values to floats

### Example Workflow
```python
config = {
    "constraints": [
        {
            "file": "data/age.csv",
            "constraint_prefix": "age%",
            "geography_column": "GEO_CODE"
        }
    ]
}
labels, targets = build_constraint_arrays(config)

#### **Key Points**
- **`np.hstack`**: Short for "horizontal stack," it concatenates arrays column-wise.  
  Example:
  ```python
  import numpy as np
  a = np.array([[1, 2], [3, 4]])
  b = np.array([[5], [6]])
  np.hstack([a, b])  # Result: [[1, 2, 5], [3, 4, 6]]
  ```

In [37]:
def encode_microdata(config, constraint_labels):
    """
    Encode microdata into a one-hot-like array where missing values are 0.
    Returns:
        microdata_encoded: np.array shape (n_individuals, n_constraints)
        ids: list of IDs from the microdata
    """
    # Step 1: Load microdata from CSV
    with open(config["microdata"]["file"], mode='r') as f:
        reader = csv.DictReader(f)  # Reads header and rows as dictionaries
        microdata = list(reader)    # Convert to list of dicts

    n_individuals = len(microdata)
    n_constraints = len(constraint_labels)

    # Step 2: Create label-to-index mapping
    label_to_idx = {label: idx for idx, label in enumerate(constraint_labels)}

    # Step 3: Initialize output array (all zeros)
    microdata_encoded = np.zeros((n_individuals, n_constraints), dtype=np.int8)

    # Step 4: Extract IDs
    ids = [row[config["microdata"]["id_column"]] for row in microdata]

    # Step 5: Encode each constraint
    for constraint in config["constraints"]:
        col = constraint["microdata_id"]
        prefix = constraint["constraint_prefix"]

        for row_idx, row in enumerate(microdata):
            value = row.get(col)  # Get value for the current constraint column

            # Skip missing values (leave as 0)
            if value is not None and value.strip() != '':  # Check for non-empty strings
                label = f"{prefix}{value}"
                if label in label_to_idx:  # Ensure label exists in constraints
                    microdata_encoded[row_idx, label_to_idx[label]] = 1

    return {
        "microdata_encoded":microdata_encoded, 
        "ids":ids
    }

In [38]:
microdata_dict = encode_microdata(config,constraints_dict["constraint_labels"])

##  `encode_microdata(config, constraint_labels)`

**Purpose**:  
Converts individual microdata records into a binary matrix matching census constraints, with automatic handling of missing values.

### Inputs
- `config` (dict): Configuration dictionary with microdata file path
- `constraint_labels` (List[str]): Pre-generated labels from `build_constraint_arrays()`

### Returns
- `microdata_encoded` (np.array): Binary matrix where:
  - Rows = Individuals
  - Columns = Constraint categories
  - Values = `1` (present) or `0` (missing/not applicable)

### Key Features
-   **Smart Encoding**:
  - Converts categorical values (e.g., `"m"`, `"25-34"`) to binary flags
  - Preserves relationships between original data and constraint categories
-   **Missing Data Handling**:
  - Empty/missing values remain `0` by default
  - Silent skipping of undefined categories
-   **Efficient Construction**:
  - Pre-allocates NumPy array for performance
  - Uses memory-efficient `int8` dtype

### Example Transformation
**Input Microdata**:
```csv
ID,sex,age
1,m,25-34
2,f,55-64
3,,45-54

In [39]:
np.savetxt('testdata/microdata_encoded.csv', microdata_dict["microdata_encoded"], delimiter=',')
np.savetxt('testdata/constraint_targets.csv', constraints_dict["constraint_targets"], delimiter=',')

In [40]:
class ParallelAnnealingGenerator:
    def __init__(self, target_distribution, population_size, micro_data, 
                 initial_temp=100.0, cooling_rate=0.99):
        """
        Parallel-safe implementation with proper annealing controls
        
        Args:
            target_distribution: 1D np.array of target category counts
            population_size: Number of individuals to select
            micro_data: Shared read-only microdata array
            initial_temp: Starting temperature (default 100.0)
            cooling_rate: Geometric cooling rate (default 0.99)
        """
        self.target = target_distribution
        self.pop_size = int(population_size)
        self.full_data = micro_data  # Shared read-only reference
        
        # Annealing controls
        self.temp = float(initial_temp)
        self.base_temp = float(initial_temp)
        self.cooling = float(cooling_rate)
        
        # Initialize population (protected copy)
        self.indices = np.random.choice(len(micro_data), size=self.pop_size, replace=True)
        self.local_pop = micro_data[self.indices].copy()  # Isolated working copy
        self.counts = self.local_pop.sum(axis=0)
        
        # Tracking
        self.best_indices = self.indices.copy()
        self.best_counts = self.counts.copy()
        self.best_error = self._calculate_error(self.counts)

    def _calculate_error(self, counts):
        """Compute L1 distance between current and target distributions"""
        return np.sum(np.abs(counts - self.target))

    def optimization_step(self):
        # 1. Select replacement candidate
        replace_pos = np.random.randint(self.pop_size)
        old_features = self.local_pop[replace_pos]
        
        # 2. Get new candidate safely from shared array
        new_idx = np.random.randint(len(self.full_data))
        new_features = self.full_data[new_idx]  # Atomic read
        
        # 3. Calculate delta
        delta = new_features - old_features
        new_counts = self.counts + delta
        new_error = self._calculate_error(new_counts)
        error_delta = new_error - self.best_error
        
        # 4. Metropolis acceptance
        if error_delta < 0 or np.random.rand() < np.exp(-error_delta / self.temp):
            self.counts = new_counts
            self.local_pop[replace_pos] = new_features
            self.indices[replace_pos] = new_idx
            
            if new_error < self.best_error:
                self.best_error = new_error
                self.best_indices = self.indices.copy()
                self.best_counts = self.counts.copy()
        
        # 5. Apply cooling
        self.temp *= self.cooling
        return self.best_error

    def reset_annealing(self, new_temp=None):
        """Reset temperature for new runs"""
        self.temp = float(new_temp) if new_temp else self.base_temp
        # Optional: Keep best solution or re-randomize
        # self.indices = self.best_indices.copy()

    def get_results(self):
        return {
            'selected_indices': self.best_indices,
            'population_counts': self.best_counts,
            'target_counts': self.target,
            'error': self.best_error,
            'final_temperature': self.temp
        }


In [41]:

# Usage pattern matching your example:
print("start")
for i in range(len(constraints_dict["geography_codes"])):
    # Initialize generator for this geography
    generator = ParallelAnnealingGenerator(
        constraints_dict["constraint_targets"][i],
        constraints_dict["population_totals"][i],  # Note: Fixed typo from your original
        microdata_dict["microdata_encoded"],
        100,  # Initial temp
        0.99   # Cooling rate
    )
    
    # Initial error threshold (10% of first step)
    threshold = generator.optimization_step() / 10
    
    # Run until convergence or max iterations
    for c in range(20000):
        current_error = generator.optimization_step()
        if current_error <= threshold :#or generator.temp <0.000001:
            break
    
    results = generator.get_results()
    # print(f"Geography {constraints_dict['geography_codes'][i]}:")
    # print(f"- Final error: {results['error']:.1f}")
    # print(f"pop: {len(results['selected_indices'])} - Sample indices: {results['selected_indices'][:5]} ")
print("done")

start
done


In [43]:
#  # For progress tracking

# def init_worker(shared_mem_name, shape, dtype):
#     """Initialize worker with shared memory"""
#     global shared_microdata
#     shm = shared_memory.SharedMemory(name=shared_mem_name)
#     shared_microdata = np.ndarray(shape, dtype=dtype, buffer=shm.buf)

# def worker(args):
#     geo_idx, target, pop_size = args
#     try:
#         # Initialize with more aggressive cooling
#         generator = ParallelAnnealingGenerator(
#             target_distribution=target,
#             population_size=pop_size,
#             micro_data=shared_microdata,
#             initial_temp=50.0,  # Lower initial temp
#             cooling_rate=0.95    # Faster cooling
#         )
        
#         # Dynamic threshold based on population size
#         threshold = max(5, pop_size * 0.01)  # At least 5, max 1% of pop size
        
#         # Optimized stopping conditions
#         for step in range(20000):
#             print(step,geo_idx)
#             error = generator.optimization_step()
#             if error <= threshold or generator.temp < 0.1:
#                 break
                
#         return {
#             'geo_idx': geo_idx,
#             'error': error,
#             'steps': step,
#             'final_temp': generator.temp
#         }
#     except Exception as e:
#         print(f"Error in geography {geo_idx}: {str(e)}")
#         return None

# # Main execution
# if __name__ == "__main__":
#     # Prepare shared memory (10-20% faster than automatic pickling)
#     microdata = microdata_dict["microdata_encoded"]
#     shm = shared_memory.SharedMemory(create=True, size=microdata.nbytes)
#     shared_array = np.ndarray(microdata.shape, dtype=microdata.dtype, buffer=shm.buf)
#     shared_array[:] = microdata[:]

#     try:
#         # Optimal worker count (leave 1 core free)
#         n_workers = max(1, os.cpu_count() - 1)
#         print(f"Using {n_workers} workers with shared memory")
        
#         with Pool(
#             processes=n_workers,
#             initializer=init_worker,
#             initargs=(shm.name, microdata.shape, microdata.dtype)
#         ) as pool:
            
#             # Chunk tasks for better load balancing
#             tasks = [
#                 (i, constraints_dict["constraint_targets"][i], 
#                 constraints_dict["population_totals"][i])
#                 for i in range(len(constraints_dict["geography_codes"]))
#             ]
            
#             # Process with progress bar
#             results = []
#             for result in tqdm(pool.imap_unordered(worker, tasks, chunksize=4), 
#                             total=len(tasks)):
#                 if result:
#                     results.append(result)
                    
#     finally:
#         shm.close()
#         shm.unlink()
    
#     # Analyze results
#     avg_steps = np.mean([r['steps'] for r in results])
#     print(f"Completed {len(results)} geographies (avg {avg_steps:.0f} steps each)")
#     print(f"Final errors: {[r['error'] for r in results[:5]]}...")

In [ ]:
results